In [1]:
import toml

import pandas as pd
import pyspark
import dxpy
import dxdata
import hail as hl

In [ ]:
with open("../config.toml") as f:
    conf = toml.load(f)

GENE = conf["GENE"]
REFERENCE_GENOME = conf["REFERENCE_GENOME"]

In [ ]:
# Spark and Hail
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

hl.init(sc=sc, default_reference=REFERENCE_GENOME, log=LOG_FILE)